_ _ del _ _

called when all references to the object is done

In [1]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value # to count referencec 

In [2]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Person {self.name}"

    def __del__(self):
        print(f"__del__ called from {self}...")

In [3]:
p = Person("Alex")

In [5]:
id_p = id(p)
ref_count(id_p)

1

In [6]:
p = None 

__del__ called from Person Alex...


In [26]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Person {self.name}"

    def __del__(self):
        print(f"__del__ called from {self}...")

    def gen_ex(self):
        raise ValueError(f"{self.name}  Something gone wrong...")

In [32]:
p = Person("Nik")
p_id = id(p)
ref_count(p_id)  # Why does __del__ was called

__del__ called from Person Nik...


1

In [33]:
try:
    p.gen_ex()
except ValueError as ex:
    error = ex
    print(ex)

NikSomething gone wrong...


In [34]:
error

ValueError('NikSomething gone wrong...')

In [35]:
ref_count(p_id)

2

In [36]:
type(error)

ValueError

In [37]:
dir(error)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'args',
 'with_traceback']

In [39]:
dir(error.__traceback__)

['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next']

In [40]:
dir(error.__traceback__.tb_frame)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'f_back',
 'f_builtins',
 'f_code',
 'f_globals',
 'f_lasti',
 'f_lineno',
 'f_locals',
 'f_trace',
 'f_trace_lines',
 'f_trace_opcodes']

In [41]:
dir(error.__traceback__.tb_frame.f_locals)

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [42]:
type(error.__traceback__.tb_frame.f_locals)

dict

In [46]:
for key, value in error.__traceback__.tb_frame.f_locals.items():
    print(key, value)

__name__ __main__
__doc__ Automatically created module for IPython interactive environment
__package__ None
__loader__ None
__spec__ None
__builtin__ <module 'builtins' (built-in)>
__builtins__ <module 'builtins' (built-in)>
_ih ['', 'import ctypes\n\ndef ref_count(address):\n    return ctypes.c_long.from_address(address).value # to count referencec ', 'class Person:\n    def __init__(self, name):\n        self.name = name\n\n    def __repr__(self):\n        return f"Person {self.name}"\n\n    def __del__(self):\n        print(f"__del__ called from {self}...")', 'p = Person("Alex")', 'id_p = id(p)', 'id_p = id(p)\nref_count(id_p)', 'p = None ', 'class Person:\n    def __init__(self, name):\n        self.name = name\n\n    def __repr__(self):\n        return f"Person {self.name}"\n\n    def __del__(self):\n        print(f"__del__ called from {self}...")\n\n    def gen_ex(self):\n        raise ValueError("Something gone wrong...")', 'p = Person("nik")\np_id = id (p)', 'p = Person("nik")\

In [47]:
for key, value in error.__traceback__.tb_frame.f_locals.items():
    if isinstance(value, Person):
        print(key, value, id(value))

p Person Nik 1783403172848


In [48]:
ref_count(p_id)

2

In [49]:
del p  # and why it is not called (__del__), means enexpected

In [50]:
del error

In [51]:
ref_count(p_id)

1

In [53]:
class Person:
    def __del__(self):
        raise ValueError("Something went bump...")

In [56]:
p =Person()

In [57]:
del p  # ignored 
print("All good, no exceptiond")

Exception ignored in: <function Person.__del__ at 0x0000019F3B939FC0>
Traceback (most recent call last):
  File "C:\Users\nikit\AppData\Local\Temp\ipykernel_4684\2199438697.py", line 3, in __del__
ValueError: Something went bump...


All good, no exceptiond


In [58]:
import sys

In [59]:
sys.stderr, sys.stdout

(<ipykernel.iostream.OutStream at 0x19f39e08e20>,
 <ipykernel.iostream.OutStream at 0x19f39e08df0>)

In [60]:
class ErrToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stderr = sys.stderr

    def __enter__(self):
        self._file = open(self._fname, "w")
        sys.stderr = self._file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stderr = self._current_stderr
        if self._file:
            self._file.close()
        return False

In [61]:
p = Person()

In [62]:
with ErrToFile("4-60err.txt"):
    del p 
    print('100')
print(200)

100
200


In [63]:
with open("4-60err.txt") as f:
    print(f.readlines())

['Exception ignored in: <function Person.__del__ at 0x0000019F3B939FC0>\n', 'Traceback (most recent call last):\n', '  File "C:\\Users\\nikit\\AppData\\Local\\Temp\\ipykernel_4684\\2199438697.py", line 3, in __del__\n', 'ValueError: Something went bump...\n']
